In [110]:
!pip install yfinance requests pandas numpy matplotlib statsmodels scipy pmdarima --quiet

In [111]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from scipy.stats import norm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

btc = yf.download(
    "BTC-USD",
    period="2y",
    progress=False,
    auto_adjust=False       
)["Close"]

btc.to_csv("btc_historical.csv")
print(f"Downloaded {len(btc)} days of BTC data")
print(f"Latest price: ${btc.iloc[-1].item():,.2f}")

Downloaded 732 days of BTC data
Latest price: $86,896.38


In [112]:
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
params = {"limit": 1000, "status": "open", "series_ticker": "KXBTCD"}
response = requests.get(url, params=params)

kalshi_strikes = []

if response.status_code == 200:
    markets = response.json().get("markets", [])
    
    for market in markets:
        ticker = market.get("ticker", "")
        if "KXBTCD" in ticker and market.get("floor_strike"):
            strike = market.get("floor_strike")
            yes_bid = market.get("yes_bid", 0) / 100
            
            if strike and yes_bid > 0:
                kalshi_strikes.append((int(strike), yes_bid))
    
    if kalshi_strikes:
        kalshi_strikes.sort()
        kalshi_strikes = kalshi_strikes[:20]
        print("API gave data")
        print(f"Retrieved {len(kalshi_strikes)} strikes")
    else:
        print("No valid strikes found")
else:
    print(f"API error (status {response.status_code})")
    

API gave data
Retrieved 20 strikes


In [113]:
def calculate_momentum_features(prices):
    features = {}
    
    features['mom_3d'] = (prices.iloc[-1] / prices.iloc[-4] - 1) if len(prices) >= 4 else 0
    features['mom_7d'] = (prices.iloc[-1] / prices.iloc[-8] - 1) if len(prices) >= 8 else 0
    features['mom_14d'] = (prices.iloc[-1] / prices.iloc[-15] - 1) if len(prices) >= 15 else 0
    
    returns = prices.pct_change().dropna()
    features['vol_7d'] = returns.tail(7).std() if len(returns) >= 7 else 0
    features['vol_30d'] = returns.tail(30).std() if len(returns) >= 30 else 0
    
    ma_7 = prices.tail(7).mean()
    ma_30 = prices.tail(30).mean()
    features['ma_ratio'] = ma_7 / ma_30 - 1 if len(prices) >= 30 else 0
    return features

In [116]:
pred, std = momentum_forecast(btc, horizon_hours=24)

ci_low = pred - 1.96 * std
ci_high = pred + 1.96 * std

print("\nMOMENTUM INDICATORS")
print("=" * 60)
features = calculate_momentum_features(btc)

print(f"3-day momentum:    {features['mom_3d'].item():>+8.2%}")
print(f"7-day momentum:    {features['mom_7d'].item():>+8.2%}")
print(f"14-day momentum:   {features['mom_14d'].item():>+8.2%}")
print(f"7d/30d MA ratio:   {features['ma_ratio'].item():>+8.2%}")
print(f"30-day volatility: {features['vol_30d'].item():>8.2%}")

print("\nPREDICTION FOR NEXT 24H")
print("=" * 60)
print(f"→ Current price  : ${btc.iloc[-1].item():,.0f}")
print(f"→ Predicted price: ${pred:,.0f}")
print(f"→ 95% interval   : ${ci_low.item():,.0f} – ${ci_high.item():,.0f}")
print(f"→ Implied change : {(pred/btc.iloc[-1].item() - 1)*100:+.2f}%")
print(f"→ Uncertainty    : ±${std.item():,.0f}")

def backtest_momentum(data, test_days=30):
    errors = []
    predictions = []
    actuals = []
    
    for i in range(test_days):
        historical = data[:-(test_days-i)]
        actual = data.iloc[-(test_days-i)].item()
        
        pred, _ = momentum_forecast(historical)
        
        error = abs(pred - actual) / actual * 100
        errors.append(error)
        predictions.append(pred)
        actuals.append(actual)
    
    return errors, predictions, actuals

print("\nBACKTEST RESULTS (Last 30 days)")
print("=" * 60)
errors, preds, actuals = backtest_momentum(btc, test_days=30)
print(f"Mean Absolute Error:  {np.mean(errors):.2f}%")
print(f"Median Error:         {np.median(errors):.2f}%")
print(f"Best Prediction:      {np.min(errors):.2f}%")
print(f"Worst Prediction:     {np.max(errors):.2f}%")
print(f"Within ±5% accuracy:  {sum(e < 5 for e in errors)}/{len(errors)} days")


MOMENTUM INDICATORS
3-day momentum:      -5.64%
7-day momentum:     -12.84%
14-day momentum:    -14.22%
7d/30d MA ratio:    -10.83%
30-day volatility:    2.25%

PREDICTION FOR NEXT 24H
→ Current price  : $86,896
→ Predicted price: $75,802
→ 95% interval   : $71,962 – $79,642
→ Implied change : -12.77%
→ Uncertainty    : ±$1,959

BACKTEST RESULTS (Last 30 days)
Mean Absolute Error:  4.48%
Median Error:         3.68%
Best Prediction:      0.20%
Worst Prediction:     12.10%
Within ±5% accuracy:  20/30 days


In [117]:
print("\nBITCOIN BET RECOMMENDATIONS")
print("=" * 85)
print(f"{'Strike':>8}  {'Model Prob':>10}  {'Market Prob':>11}  {'Edge':>8}  Recommendation")
print("-" * 85)

for strike, market_prob in kalshi_strikes:
    model_prob = 1 - norm.cdf(strike, pred, std)
    edge = model_prob - market_prob
    
    rec = "BET YES" if edge.item() > 0.05 else "BET NO" if edge.item() < -0.05 else "NO EDGE"
    
    print(f"${strike:>6,}    {model_prob.item():>8.1%}      {market_prob:>8.1%}    {edge.item():+6.1%}    {rec}")


BITCOIN BET RECOMMENDATIONS
  Strike  Model Prob  Market Prob      Edge  Recommendation
-------------------------------------------------------------------------------------
$81,249        0.3%         95.0%    -94.7%    BET NO
$81,749        0.1%         95.0%    -94.9%    BET NO
$81,999        0.1%         95.0%    -94.9%    BET NO
$82,249        0.1%         95.0%    -94.9%    BET NO
$82,249        0.1%         95.0%    -94.9%    BET NO
$82,499        0.0%         95.0%    -95.0%    BET NO
$82,749        0.0%         94.0%    -94.0%    BET NO
$82,749        0.0%         95.0%    -95.0%    BET NO
$82,999        0.0%         95.0%    -95.0%    BET NO
$83,249        0.0%         93.0%    -93.0%    BET NO
$83,249        0.0%         95.0%    -95.0%    BET NO
$83,499        0.0%         95.0%    -95.0%    BET NO
$83,749        0.0%         92.0%    -92.0%    BET NO
$83,749        0.0%         95.0%    -95.0%    BET NO
$83,999        0.0%         95.0%    -95.0%    BET NO
$84,249        